In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import sys

sys.path.append(os.path.abspath("../"))
from dataset import DS_NAMES
from dataset import DS_DIMENSIONS
from utilities.figure_tools import get_latest_results_directory, make_fig_directory, load_results

cmap = plt.get_cmap("Set1")
colors = cmap.colors

results_path = "../results/turing/compare_methods_softaxilfNORF.csv"
output_dir = "./compare-methods/"
export_figures = False

if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [2]:
all_results = pd.read_csv(results_path)
all_results = all_results[all_results["n_trees"] == 10]
all_results = all_results.groupby(["dataset", "explainer"]).mean().reset_index()
print("explainers:", list(all_results["explainer"].unique()))
print("datasets", list(all_results["dataset"].unique()))
all_results.head(100)

explainers: []
datasets []


,dataset,explainer,n_trees,max_depth,n_rects,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain


In [3]:
df = all_results.set_index(["dataset", "explainer"])
metrics = ["sample_time", "avg_dist", "per_valid"]

with open("compare_methods_table.csv", "w") as f:
    # header row one
    f.write("dataset,")
    for expl in all_results["explainer"].unique():
        for m in metrics:
            f.write(expl + ",")
    f.write("\n")
    # header row two
    f.write(",")
    for expl in all_results["explainer"].unique():
        for m in metrics:
            f.write(m + ",")
    f.write("\n")
    # body row
    for ds in all_results["dataset"].unique():
        f.write(ds + ",")
        for expl in all_results["explainer"].unique():
            for m in metrics:
                f.write(str(df.loc[ds, expl][m]) + ",")
        f.write("\n")

In [4]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["per_valid"])

dataset


In [5]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["avg_dist"])

dataset


In [6]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["sample_time"])

dataset


In [7]:
all_results = pd.read_csv(results_path)
all_results = all_results[all_results["n_trees"] == 100]
# all_results["diff"] = all_results["sample_time"] - (((all_results["sample_time"] * all_results["n_explain"]) - ((1 - all_results["per_valid"]) * all_results["n_explain"] * 300)) / all_results["n_explain"])
tot_time = all_results["sample_time"] * all_results["n_explain"]
n_timeout = all_results["per_valid"] * all_results["n_explain"]
max_time = 300
timeout_cost = n_timeout * max_time
all_results["adj_time"] = (tot_time - timeout_cost) / all_results["n_explain"]
# all_results = all_results.groupby(["dataset", "explainer"]).mean().reset_index()
# print("explainers:", list(all_results["explainer"].unique()))
# print("datasets", list(all_results["dataset"].unique()))
all_results.head()

,dataset,explainer,n_trees,max_depth,n_rects,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain,adj_time
0,cancer,FACETIndex,100,NaN,100,0,0.947368,0.951220,0.906977,0.928571,1.0,0.436970,15.80,0.180966,0.056400,0.002820,20,-299.997180
1,glass,FACETIndex,100,NaN,100,0,0.848485,0.850000,0.894737,0.871795,1.0,0.254132,5.85,0.151536,0.025647,0.001282,20,-299.998718
2,magic,FACETIndex,100,NaN,100,0,0.873817,0.878870,0.931501,0.904421,1.0,0.317976,9.20,4.731014,0.027940,0.001397,20,-299.998603
3,spambase,FACETIndex,100,NaN,100,0,0.958696,0.950262,0.950262,0.950262,1.0,0.185882,22.10,1.125375,0.043892,0.002195,20,-299.997805
4,vertebral,FACETIndex,100,NaN,100,0,0.822581,0.829787,0.928571,0.876404,1.0,0.169264,4.75,0.168747,0.016347,0.000817,20,-299.999183
